# **InterUni Datathon**
# MACS Model Masters

# Section 1: EDA

## I. Explore data

In [4]:
# Import necessary packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Import data
df = pd.read_csv("train.csv")

In [6]:
df.head(2)

,TransactionNumber,UserID,Age,Gender,Occupation,EducationLevel,MaritalStatus,NumDependents,Income,Expenditure,...,MerchantID,TransactionType,TransactionLocation,DeviceType,Latitude,Longitude,EmailDomain,Terrorism,UserTenure,IsFraud
0,8765,70,37,Female,Professional,Bachelor,Widowed,3,28884.43 AUD,14610.61 AUD,...,M006,Withdrawal,Adelaide,Mobile,-31.840233,145.612793,jon44@disposable.com,False,113,1
1,9645,3386,34,Male,Student,High School,Married,4,AU$ 54919.07,39169.49 AUD,...,M002,Withdrawal,Canberra,Mobile,-37.020100,144.964600,emilyreese@gmail.com,False,104,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10966 entries, 0 to 10965
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionNumber    10966 non-null  int64  
 1   UserID               10966 non-null  int64  
 2   Age                  10966 non-null  int64  
 3   Gender               10966 non-null  object 
 4   Occupation           10966 non-null  object 
 5   EducationLevel       10966 non-null  object 
 6   MaritalStatus        10966 non-null  object 
 7   NumDependents        10966 non-null  int64  
 8   Income               10966 non-null  object 
 9   Expenditure          10966 non-null  object 
 10  GiftsTransaction     10966 non-null  object 
 11  TransactionDate      10966 non-null  object 
 12  TransactionTime      10966 non-null  object 
 13  TransactionAmount    10966 non-null  object 
 14  MerchantID           10966 non-null  object 
 15  TransactionType      10966 non-null 

In [8]:
df.describe()

,TransactionNumber,UserID,Age,NumDependents,Latitude,Longitude,UserTenure,IsFraud
count,10966.000000,10966.000000,10966.000000,10966.000000,10923.000000,10923.000000,10966.000000,10966.000000
mean,9117.500091,2483.885282,2751.845887,1.995714,-30.363255,141.254786,60.786157,0.364308
std,5293.669640,1429.402615,9730.988917,1.408035,6.962819,11.268395,34.254477,0.481257
min,1.000000,1.000000,-68.000000,0.000000,-41.640079,-112.026050,1.000000,0.000000
25%,4516.250000,1253.000000,26.000000,1.000000,-37.020100,142.702789,31.000000,0.000000
50%,9120.500000,2471.000000,34.000000,2.000000,-31.840233,144.964600,61.000000,0.000000
75%,13710.750000,3727.000000,43.000000,3.000000,-25.042261,145.612793,90.000000,1.000000
max,18277.000000,5000.000000,67000.000000,4.000000,57.851580,149.012375,119.000000,1.000000


## II. Clean data

In [10]:
# Age must be between 18 and 99
print(df.shape[0])
print(df[df["Age"] > 100].shape[0])
print(df[df["Age"] < 17].shape[0])
df = df[(df["Age"] < 100) & (df["Age"] > 17)]

10966
858
886


In [11]:
# Gender
# Clean
gender_correction = {"isnotfemale": "Male", "he": "Male", "he": "Male", "man": "Male", "man": "Male","fem": "Female", "she":"Female","woman":"Female","isnotmale":"Female"}
df['Gender']=df['Gender'].replace(gender_correction)
df["Gender"].value_counts()

# Onehot encoding
code_gender = pd.get_dummies(df[["Gender"]], dtype = int, drop_first = True)
df = df.join([code_gender])
df = df.drop(columns = "Gender").reset_index(drop = True)

In [12]:
# Occupation
# Onehot encoding
code_occupation = pd.get_dummies(df[["Occupation"]], dtype = int, drop_first = True)
df = df.join([code_occupation])
df = df.drop(columns = "Occupation").reset_index(drop = True)

In [13]:
# Education Level (ordinal value)
df["EducationLevel"] = df["EducationLevel"].replace({"High School": 0,"Bachelor":1, "Master":2,"PhD":3})

In [14]:
# Marital Status
# Onehot encoding
code_marital_status = pd.get_dummies(df[["MaritalStatus"]], dtype = int, drop_first = True)
df = df.join([code_marital_status])
df = df.drop(columns = "MaritalStatus").reset_index(drop = True)

In [15]:
# Income, Expenditure, GiftTransaction, Transaction Amount
df[["Income", "Expenditure", "GiftsTransaction", "TransactionAmount"]].head(10)

,Income,Expenditure,GiftsTransaction,TransactionAmount
0,28884.43 AUD,14610.61 AUD,£ 1050.01,AU$ 258.14
1,AU$ 54919.07,39169.49 AUD,£ 4969.71,34.94 AUD
2,AU$ 74728.57,55873.76 AUD,£ 1149.85,AU$ 323.82
3,AU$ 55712.62,AED 89649.04,AU$ 4335.7,AED 32.49
4,53004.7 AUD,AED 43601.02,4763.48 AUD,1140.75 AED
5,101381.56 AUD,AED 81036.3,196.29 AUD,67.39 AED
6,AU$ 63035.98,91610.14 AED,3009.44 AUD,899.6 AED
7,AU$ 61284.85,AU$ 36604.93,1214.27 GBP,19.41 AUD
8,AU$ 28419.25,AU$ 11172.93,600.92 GBP,AU$ 111.73
9,57384.51 AUD,AU$ 40721.71,10819.52 GBP,110.54 AUD


In [16]:
# Income, Expenditure, GiftsTransaction, TransactionAmount

# GiftsTransaction
df[['first_value', 'second_value']] = df["GiftsTransaction"].str.split(' ', expand=True)
    
df["first_value_len"] = df["first_value"].apply(lambda s:len(s))
df["second_value_len"] = df["second_value"].apply(lambda s:len(s))
    
    # function to get monetary symbol
def get_monetary_symbol(row):
    if row['first_value_len'] == 1:
        return row['first_value']
    elif row['first_value'] == "AU$":
        return row['first_value']
    elif row['first_value'] == "AED":
        return row['first_value']
    elif row['second_value'] == "GBP":
        return row['second_value']
    elif row['second_value'] == "AUD":
        return row['second_value']
    else:
        return None
    
df["monetary_symbol"] = df.apply(get_monetary_symbol, axis=1)
    
    # function to get monetary value
def get_monetary_value(row):
    if row['first_value'] == row["monetary_symbol"]:
        return row['second_value']
    else:
        return row['first_value']
            
df["monetary_value"] = df.apply(get_monetary_value, axis=1)

df["monetary_value"] = df["monetary_value"].astype("float")
    
    # function to convert to AUD
def get_exchange_rate(row):
    if row['monetary_symbol'] == "£":
        return 1.96
    elif row['monetary_symbol'] == "GBP":
        return 1.96
    elif row['monetary_symbol'] == "AED":
        return 0.41
    else:
        return 1
    
df["exchange_rate"] = df.apply(get_exchange_rate, axis=1)
    
df["GiftsTransaction_AUD_equivalent"] = df["exchange_rate"]*df["monetary_value"]
    
df = df.drop(columns = ["first_value_len", "second_value_len", 'first_value', 'second_value', "exchange_rate", "monetary_symbol","monetary_value"]).reset_index(drop = True)

In [17]:
# Income, Expenditure, GiftsTransaction, TransactionAmount

# Income
df[['first_value', 'second_value']] = df["Income"].str.split(' ', expand=True)
    
df["first_value_len"] = df["first_value"].apply(lambda s:len(s))
df["second_value_len"] = df["second_value"].apply(lambda s:len(s))
    
    # function to get monetary symbol
def get_monetary_symbol(row):
    if row['first_value_len'] == 1:
        return row['first_value']
    elif row['first_value'] == "AU$":
        return row['first_value']
    elif row['first_value'] == "AED":
        return row['first_value']
    elif row['second_value'] == "GBP":
        return row['second_value']
    elif row['second_value'] == "AUD":
        return row['second_value']
    else:
        return None
    
df["monetary_symbol"] = df.apply(get_monetary_symbol, axis=1)
    
    # function to get monetary value
def get_monetary_value(row):
    if row['first_value'] == row["monetary_symbol"]:
        return row['second_value']
    else:
        return row['first_value']
            
df["monetary_value"] = df.apply(get_monetary_value, axis=1)

df["monetary_value"] = df["monetary_value"].astype("float")
    
    # function to convert to AUD
def get_exchange_rate(row):
    if row['monetary_symbol'] == "£":
        return 1.96
    elif row['monetary_symbol'] == "GBP":
        return 1.96
    elif row['monetary_symbol'] == "AED":
        return 0.41
    else:
        return 1
    
df["exchange_rate"] = df.apply(get_exchange_rate, axis=1)
    
df["Income_AUD_equivalent"] = df["exchange_rate"]*df["monetary_value"]
    
df = df.drop(columns = ["first_value_len", "second_value_len", 'first_value', 'second_value', "exchange_rate", "monetary_symbol","monetary_value"]).reset_index(drop = True)

In [18]:
# Income, Expenditure, GiftsTransaction, TransactionAmount

# Expenditure
df[['first_value', 'second_value']] = df["Expenditure"].str.split(' ', expand=True)
    
df["first_value_len"] = df["first_value"].apply(lambda s:len(s))
df["second_value_len"] = df["second_value"].apply(lambda s:len(s))
    
    # function to get monetary symbol
def get_monetary_symbol(row):
    if row['first_value_len'] == 1:
        return row['first_value']
    elif row['first_value'] == "AU$":
        return row['first_value']
    elif row['first_value'] == "AED":
        return row['first_value']
    elif row['second_value'] == "GBP":
        return row['second_value']
    elif row['second_value'] == "AUD":
        return row['second_value']
    else:
        return None
    
df["monetary_symbol"] = df.apply(get_monetary_symbol, axis=1)
    
    # function to get monetary value
def get_monetary_value(row):
    if row['first_value'] == row["monetary_symbol"]:
        return row['second_value']
    else:
        return row['first_value']
            
df["monetary_value"] = df.apply(get_monetary_value, axis=1)

df["monetary_value"] = df["monetary_value"].astype("float")
    
    # function to convert to AUD
def get_exchange_rate(row):
    if row['monetary_symbol'] == "£":
        return 1.96
    elif row['monetary_symbol'] == "GBP":
        return 1.96
    elif row['monetary_symbol'] == "AED":
        return 0.41
    else:
        return 1
    
df["exchange_rate"] = df.apply(get_exchange_rate, axis=1)
    
df["Expenditure_AUD_equivalent"] = df["exchange_rate"]*df["monetary_value"]
    
df = df.drop(columns = ["first_value_len", "second_value_len", 'first_value', 'second_value', "exchange_rate", "monetary_symbol","monetary_value"]).reset_index(drop = True)

In [19]:
# Income, Expenditure, GiftsTransaction, TransactionAmount

# TransactionAmount
df[['first_value', 'second_value']] = df["TransactionAmount"].str.split(' ', expand=True)
    
df["first_value_len"] = df["first_value"].apply(lambda s:len(s))
df["second_value_len"] = df["second_value"].apply(lambda s:len(s))
    
    # function to get monetary symbol
def get_monetary_symbol(row):
    if row['first_value_len'] == 1:
        return row['first_value']
    elif row['first_value'] == "AU$":
        return row['first_value']
    elif row['first_value'] == "AED":
        return row['first_value']
    elif row['second_value'] == "GBP":
        return row['second_value']
    elif row['second_value'] == "AUD":
        return row['second_value']
    else:
        return None
    
df["monetary_symbol"] = df.apply(get_monetary_symbol, axis=1)
    
    # function to get monetary value
def get_monetary_value(row):
    if row['first_value'] == row["monetary_symbol"]:
        return row['second_value']
    else:
        return row['first_value']
            
df["monetary_value"] = df.apply(get_monetary_value, axis=1)

df["monetary_value"] = df["monetary_value"].astype("float")
    
    # function to convert to AUD
def get_exchange_rate(row):
    if row['monetary_symbol'] == "£":
        return 1.96
    elif row['monetary_symbol'] == "GBP":
        return 1.96
    elif row['monetary_symbol'] == "AED":
        return 0.41
    else:
        return 1
    
df["exchange_rate"] = df.apply(get_exchange_rate, axis=1)
    
df["TransactionAmount_AUD_equivalent"] = df["exchange_rate"]*df["monetary_value"]
    
df = df.drop(columns = ["first_value_len", "second_value_len", 'first_value', 'second_value', "exchange_rate", "monetary_symbol","monetary_value"]).reset_index(drop = True)

In [20]:
df = df.drop(columns = ["GiftsTransaction", "Income", 'Expenditure', 'TransactionAmount']).reset_index(drop = True)

In [21]:
# Transaction date, Transaction time
df["TransactionDate"] =pd.to_datetime(df["TransactionDate"])

In [22]:
# Transaction Type
code_transaction_type = pd.get_dummies(df[["TransactionType"]], dtype = int, drop_first = True)
df = df.join([code_transaction_type])
df = df.drop(columns = "TransactionType").reset_index(drop = True)

In [23]:
df["TransactionLocation"]=df["TransactionLocation"].str.upper()

mapping = {"MELB":"MELBOURNE", "MEL":"MELBOURNE", "MLB":"MELBOURNE", "MELBURN" : "MELBOURNE",
           "BNE":"BRISBANE", 
           "DRW": "DARWIN",
           "CBR":"CANBERRA",
           "PTH":"PERTH",
           "SYD":"SYDNEY",
           "ADL":"ADELAIDE", "ADELAIDE CITY":"ADELAIDE",
           "HBT":"HOBART"}

df["TransactionLocation"] = df["TransactionLocation"].replace(mapping)

code_TransactionLocation = pd.get_dummies(df[["TransactionLocation"]], dtype = int, drop_first = True)
df = df.join([code_TransactionLocation])
df = df.drop(columns = "TransactionLocation").reset_index(drop = True)

In [24]:
# Device Type
df["DeviceType"] = df["DeviceType"].replace({"galaxys7": "Mobile","iphone 15":"Mobile", "android":"Mobile","smartphone":"Mobile","mob":"Mobile"})

code_DeviceType = pd.get_dummies(df[["DeviceType"]], dtype = int, drop_first = True)
df = df.join([code_DeviceType])
df = df.drop(columns = "DeviceType").reset_index(drop = True)

In [25]:
# Lat Long
df = df.dropna()

## III. Analyze data

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9188 entries, 0 to 9221
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   TransactionNumber                 9188 non-null   int64         
 1   UserID                            9188 non-null   int64         
 2   Age                               9188 non-null   int64         
 3   EducationLevel                    9188 non-null   int64         
 4   NumDependents                     9188 non-null   int64         
 5   TransactionDate                   9188 non-null   datetime64[ns]
 6   TransactionTime                   9188 non-null   object        
 7   MerchantID                        9188 non-null   object        
 8   Latitude                          9188 non-null   float64       
 9   Longitude                         9188 non-null   float64       
 10  EmailDomain                       9188 non-null   obj

In [28]:
df_final = df.drop(columns = ["TransactionNumber", "UserID", "TransactionDate", "TransactionTime", "MerchantID", "Latitude",
                              "Longitude", "EmailDomain"]).reset_index(drop = True)

In [29]:
corr = df_final.corr()
corr.style.background_gradient(cmap='coolwarm')

,Age,EducationLevel,NumDependents,Terrorism,UserTenure,IsFraud,Gender_Male,Occupation_Retired,Occupation_Student,Occupation_Unemployed,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widowed,GiftsTransaction_AUD_equivalent,Income_AUD_equivalent,Expenditure_AUD_equivalent,TransactionAmount_AUD_equivalent,TransactionType_Purchase,TransactionType_Transfer,TransactionType_Withdrawal,TransactionLocation_BRISBANE,TransactionLocation_CANBERRA,TransactionLocation_DARWIN,TransactionLocation_HOBART,TransactionLocation_MELBOURNE,TransactionLocation_PERTH,TransactionLocation_SYDNEY,DeviceType_Mobile,DeviceType_Tablet
Age,1.000000,0.011660,-0.005134,0.011183,0.014564,-0.004305,-0.006213,0.003669,-0.015621,0.011990,-0.023194,0.025944,0.003713,-0.001839,-0.008210,0.007455,0.020920,0.001284,-0.004556,-0.000506,0.015619,0.004470,0.008555,0.000856,0.000784,-0.002795,-0.014922,0.011757,-0.010756
EducationLevel,0.011660,1.000000,-0.047779,-0.000103,0.031220,0.002895,0.009016,-0.006758,-0.002198,0.013553,0.009271,-0.010072,0.020739,-0.035130,0.002111,0.014540,0.002069,0.009562,-0.002921,-0.003504,0.009755,-0.009509,-0.008398,-0.003308,0.007676,0.000091,-0.002390,0.005702,-0.003525
NumDependents,-0.005134,-0.047779,1.000000,-0.000394,-0.011637,-0.026485,-0.005209,-0.001977,0.017185,0.010635,-0.003874,0.014886,-0.000843,0.003112,-0.018355,-0.017460,-0.008716,0.002914,0.005990,-0.005145,0.004982,0.001046,-0.015876,0.014163,0.002688,-0.005623,-0.004563,-0.012534,0.013031
Terrorism,0.011183,-0.000103,-0.000394,1.000000,0.010254,0.012093,0.044919,-0.015967,-0.014212,-0.002419,-0.016741,0.016919,0.008930,0.002794,0.021278,0.000555,0.021701,0.005287,0.008969,-0.004304,-0.003656,-0.003176,0.012160,-0.014850,0.002142,0.007011,0.007949,0.002451,0.014742
UserTenure,0.014564,0.031220,-0.011637,0.010254,1.000000,0.000863,0.021081,-0.014434,0.001393,-0.002043,0.007530,-0.004094,-0.022565,0.033566,-0.005351,-0.014309,0.002646,0.012429,-0.007609,-0.000076,-0.008562,0.014031,-0.011380,0.001093,0.007986,-0.002429,0.002772,0.025858,-0.011848
IsFraud,-0.004305,0.002895,-0.026485,0.012093,0.000863,1.000000,-0.002945,-0.013717,0.012223,0.009280,0.002197,-0.018088,0.005871,0.011139,0.010599,0.003053,0.114979,-0.278498,-0.135511,0.558740,0.035292,0.030401,0.026817,0.041940,-0.234234,0.035328,0.025958,0.010174,-0.008973
Gender_Male,-0.006213,0.009016,-0.005209,0.044919,0.021081,-0.002945,1.000000,-0.035127,0.001812,0.004103,-0.002005,0.007443,-0.007530,0.001545,-0.016346,-0.030604,0.016190,0.001823,0.009398,0.000167,0.000139,0.000992,0.021162,0.004513,-0.010104,-0.004336,-0.013103,-0.022584,0.025658
Occupation_Retired,0.003669,-0.006758,-0.001977,-0.015967,-0.014434,-0.013717,-0.035127,1.000000,-0.223085,-0.115312,-0.006010,0.027181,-0.013662,0.007825,0.017072,0.029305,-0.009233,-0.004708,0.007216,-0.001804,0.023690,-0.010955,0.005783,-0.011483,-0.017466,-0.000100,0.008397,0.011906,0.003724
Occupation_Student,-0.015621,-0.002198,0.017185,-0.014212,0.001393,0.012223,0.001812,-0.223085,1.000000,-0.225985,-0.033764,0.025201,-0.001995,-0.008061,-0.013547,-0.008743,0.021621,0.008132,-0.004716,-0.002758,0.002564,0.007265,-0.006534,-0.003012,0.007716,0.000977,-0.001566,0.011518,-0.001829
Occupation_Unemployed,0.011990,0.013553,0.010635,-0.002419,-0.002043,0.009280,0.004103,-0.115312,-0.225985,1.000000,0.043330,-0.040413,0.012313,-0.012254,0.011072,0.006127,0.011595,-0.018771,0.020725,0.002357,0.003366,0.003192,-0.010096,-0.011183,-0.000771,0.002541,0.021563,-0.001345,0.004572


## IV. Model

In [31]:
X = df_final.drop(columns = "IsFraud")
y = df_final["IsFraud"]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 8, stratify = y)

In [33]:
!pip install xgboost

In [34]:
import xgboost as xgb

In [35]:
from xgboost import XGBClassifier

# Create an instance of the XGBClassifier
xgb_classifier = XGBClassifier(objective='binary:logistic')

# Fit the model to the training data
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [36]:
# make predictions
y_pred = xgb_classifier.predict(X_test)

from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.7654530059271805